In [13]:
# vietlott_env\Scripts\activate

# Cài đặt thư viện 

In [14]:
! pip install pandas deap requests bayesian-optimization beautifulsoup4 seaborn selenium matplotlib sklearn tensorflow numpy scikit-learn xgboost lightgbm tensorflow-addons optuna imbalanced-learn torch schedule apscheduler pyarrow joblib


You should consider upgrading via the 'e:\python3_8_64bit\python.exe -m pip install --upgrade pip' command.


# Xử lý data

In [15]:
import pandas as pd
import os

# Tham số hóa đường dẫn
DATA_DIR = "data_645"
FILE_NAME = "E:/vietlot_mega\data_645/vietlott_645_filtered.csv"
FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

# Kiểm tra file tồn tại
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Không tìm thấy file: {FILE_PATH}")

# Đọc dữ liệu
try:
    df = pd.read_csv(FILE_PATH, sep="\t")
except Exception as e:
    raise Exception(f"Lỗi khi đọc file {FILE_PATH}: {e}")

# Kiểm tra cột
expected_columns = ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']
if not all(col in df.columns for col in expected_columns):
    raise ValueError(f"Dữ liệu thiếu cột: {set(expected_columns) - set(df.columns)}")

# Kiểm tra dữ liệu thiếu
if df.isna().any().any():
    print("Cảnh báo: Dữ liệu có giá trị thiếu:")
    print(df.isna().sum())

# In thông tin cơ bản
print("Thông tin dữ liệu:")
print(f"- Số cột: {len(df.columns)}")
print(f"- Tên cột: {list(df.columns)}")
print(f"- Số dòng: {len(df)}")
print(f"- Dữ liệu mẫu (5 dòng đầu):\n{df.head()}")

Thông tin dữ liệu:
- Số cột: 3
- Tên cột: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']
- Số dòng: 1356
- Dữ liệu mẫu (5 dòng đầu):
   Ngày Mở Thưởng            Kết Quả    Giải Jackpot
0  T4, 28/05/2025  17 22 23 28 31 41  29.736.250.000
1  CN, 25/05/2025  05 14 23 24 28 44  25.290.182.500
2  T6, 23/05/2025  02 04 15 16 29 43  23.051.056.000
3  T4, 21/05/2025  02 13 14 21 24 30  20.908.114.000
4  CN, 18/05/2025  06 07 17 27 30 42  18.835.213.500


In [16]:
import pandas as pd
import os

# Tham số hóa đường dẫn
DATA_DIR = "data_645"
FILE_NAME = "vietlott_645_filtered.csv"
FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

# Hàm kiểm tra tính hợp lệ của kết quả
def validate_result(result):
    try:
        if len(result) != 6:
            return False, "Phải có đúng 6 số"
        if not all(isinstance(x, int) and 1 <= x <= 45 for x in result):
            return False, "Số phải trong khoảng 1-45"
        return True, ""
    except:
        return False, "Định dạng không hợp lệ"

# Đọc dữ liệu
try:
    df = pd.read_csv(FILE_PATH, sep="\t", encoding="utf-8")
except Exception as e:
    raise Exception(f"Lỗi khi đọc file {FILE_PATH}: {e}")

# Kiểm tra và xử lý cột
if 'Kết Quả' in df.columns and 'Ngày Mở Thưởng' in df.columns and 'Giải Jackpot' in df.columns:
    # Chuyển đổi Kết Quả từ chuỗi phân cách bởi khoảng trắng sang danh sách số nguyên
    df['Kết Quả'] = df['Kết Quả'].apply(lambda x: [int(i) for i in x.split()] if isinstance(x, str) else x)
    
    # Kiểm tra tính hợp lệ của Kết Quả
    df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
    invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
    if not invalid_results.empty:
        print("Cảnh báo: Phát hiện kết quả không hợp lệ:")
        print(invalid_results[['Ngày Mở Thưởng', 'Kết Quả', 'Valid_Result']])
    
    # Xử lý cột Ngày: Loại bỏ tiền tố ngày trong tuần (CN, T6, T4) và chuyển đổi định dạng
    try:
        # Tách phần ngày (bỏ tiền tố như CN, T6, T4)
        df['Ngày Mở Thưởng'] = df['Ngày Mở Thưởng'].str.split(',').str[-1].str.strip()
        # Chuyển đổi sang định dạng datetime
        df['Ngày Mở Thưởng'] = pd.to_datetime(df['Ngày Mở Thưởng'], format='%d/%m/%Y', dayfirst=True)
    except Exception as e:
        raise ValueError(f"Lỗi khi chuyển đổi cột Ngày: {e}")
    
    # Kiểm tra Jackpot
    df['Giải Jackpot'] = pd.to_numeric(df['Giải Jackpot'], errors='coerce')
    if df['Giải Jackpot'].isna().any():
        print("Cảnh báo: Có giá trị Jackpot không hợp lệ:")
        print(df[df['Giải Jackpot'].isna()][['Ngày Mở Thưởng', 'Giải Jackpot']])
    
    # Đổi tên cột cho thống nhất
    df = df.rename(columns={'Ngày Mở Thưởng': 'Ngày', 'Giải Jackpot': 'Jackpot'})
    
    # In dữ liệu mẫu
    print("Dữ liệu sau khi xử lý (5 dòng đầu):")
    print(df[['Ngày', 'Kết Quả', 'Jackpot']].head())
else:
    raise ValueError("Dữ liệu thiếu cột cần thiết")

Cảnh báo: Có giá trị Jackpot không hợp lệ:
     Ngày Mở Thưởng  Giải Jackpot
0        2025-05-28           NaN
1        2025-05-25           NaN
2        2025-05-23           NaN
3        2025-05-21           NaN
4        2025-05-18           NaN
...             ...           ...
1351     2016-08-05           NaN
1352     2016-08-03           NaN
1353     2016-07-31           NaN
1354     2016-07-29           NaN
1355     2016-07-27           NaN

[1356 rows x 2 columns]
Dữ liệu sau khi xử lý (5 dòng đầu):
        Ngày                   Kết Quả  Jackpot
0 2025-05-28  [17, 22, 23, 28, 31, 41]      NaN
1 2025-05-25   [5, 14, 23, 24, 28, 44]      NaN
2 2025-05-23    [2, 4, 15, 16, 29, 43]      NaN
3 2025-05-21   [2, 13, 14, 21, 24, 30]      NaN
4 2025-05-18    [6, 7, 17, 27, 30, 42]      NaN


In [17]:
df.to_csv(r"E:/vietlot_mega/data_645/vietlott_645_clean.csv", index=False, encoding="utf-8-sig")
print("✅ Đã ghi file thành công: vietlott_645_clean.csv")

✅ Đã ghi file thành công: vietlott_645_clean.csv


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations
import numpy as np
import os

# Tham số hóa đường dẫn lưu biểu đồ
OUTPUT_DIR = "plots"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Kiểm tra tính hợp lệ của Kết Quả
if 'Valid_Result' not in df.columns:
    df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
if not invalid_results.empty:
    raise ValueError("Dữ liệu chứa kết quả không hợp lệ, vui lòng kiểm tra.")

# 1. Phân tích tần suất số
plt.figure(figsize=(15, 6))
all_numbers = np.concatenate(df['Kết Quả'].values)
freq = pd.Series(all_numbers).value_counts().sort_index()
sns.barplot(x=freq.index, y=freq.values, palette='viridis')
plt.title('Tần suất xuất hiện của các số (1-45)', fontsize=14)
plt.xlabel('Số', fontsize=12)
plt.ylabel('Tần suất', fontsize=12)
plt.xticks(np.arange(1, 46, 1), rotation=45, fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'number_frequency.png'), dpi=300)
plt.close()

# 2. Phân tích cặp số và bộ ba số
pair_counts = Counter()
triple_counts = Counter()
for result in df['Kết Quả']:
    pairs = list(combinations(sorted(result), 2))
    pair_counts.update(pairs)
    triples = list(combinations(sorted(result), 3))
    triple_counts.update(triples)

# Chuẩn hóa tần suất
num_draws = len(df)
top_pairs = pd.DataFrame(pair_counts.most_common(5), columns=['Cặp số', 'Tần suất'])
top_pairs['Tần suất (%)'] = top_pairs['Tần suất'] / num_draws * 100
print("\nTop 5 cặp số xuất hiện cùng nhau nhiều nhất:")
print(top_pairs[['Cặp số', 'Tần suất', 'Tần suất (%)']])

top_triples = pd.DataFrame(triple_counts.most_common(5), columns=['Bộ ba số', 'Tần suất'])
top_triples['Tần suất (%)'] = top_triples['Tần suất'] / num_draws * 100
print("\nTop 5 bộ ba số xuất hiện cùng nhau nhiều nhất:")
print(top_triples[['Bộ ba số', 'Tần suất', 'Tần suất (%)']])

# 3. Phân tích xu hướng Jackpot
plt.figure(figsize=(14, 7))
plt.plot(df['Ngày'], df['Jackpot'] / 1e9, label='Jackpot (tỷ VND)', color='blue')
plt.title('Xu hướng giá trị Jackpot theo thời gian', fontsize=14)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Jackpot (tỷ VND)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'jackpot_trend.png'), dpi=300)
plt.close()

# 4. Phân tích tăng/giảm Jackpot
df['Jackpot_Change'] = df['Jackpot'].diff()
df['Trend'] = df['Jackpot_Change'].apply(lambda x: 'Tăng' if x > 0 else 'Giảm' if x < 0 else 'Không đổi')
trend_counts = df['Trend'].value_counts()
print("\nTần suất tăng/giảm Jackpot:")
print(trend_counts)

avg_increase = df[df['Jackpot_Change'] > 0]['Jackpot_Change'].mean() / 1e9
avg_decrease = df[df['Jackpot_Change'] < 0]['Jackpot_Change'].mean() / 1e9
print(f"\nGiá trị tăng trung bình: {avg_increase:.2f} tỷ VND")
print(f"Giá trị giảm trung bình: {avg_decrease:.2f} tỷ VND")

# 5. Phân tích chu kỳ Jackpot
df['Jackpot_Reset'] = df['Jackpot_Change'] < -10e9  # Giả định giảm lớn là reset Jackpot
cycles = []
current_cycle = 0
for reset in df['Jackpot_Reset']:
    if reset:
        cycles.append(current_cycle)
        current_cycle = 0
    else:
        current_cycle += 1
if current_cycle > 0:
    cycles.append(current_cycle)

print("\nPhân tích chu kỳ Jackpot:")
print(f"- Số chu kỳ: {len(cycles)}")
print(f"- Độ dài chu kỳ trung bình: {np.mean(cycles):.2f} kỳ")
print(f"- Độ dài chu kỳ tối đa: {max(cycles)} kỳ")
print(f"- Độ dài chu kỳ tối thiểu: {min(cycles)} kỳ")

C:\Users\NITRO 5\AppData\Local\Temp\ipykernel_18208\3745541686.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=freq.index, y=freq.values, palette='viridis')



Top 5 cặp số xuất hiện cùng nhau nhiều nhất:
     Cặp số  Tần suất  Tần suất (%)
0  (10, 22)        34      2.507375
1  (13, 20)        34      2.507375
2   (7, 44)        33      2.433628
3   (4, 37)        33      2.433628
4  (19, 24)        32      2.359882

Top 5 bộ ba số xuất hiện cùng nhau nhiều nhất:
       Bộ ba số  Tần suất  Tần suất (%)
0  (11, 26, 28)         9      0.663717
1  (10, 22, 36)         9      0.663717
2    (1, 7, 16)         9      0.663717
3  (25, 35, 39)         8      0.589971
4  (10, 22, 43)         8      0.589971

Tần suất tăng/giảm Jackpot:
Trend
Không đổi    1356
Name: count, dtype: int64

Giá trị tăng trung bình: nan tỷ VND
Giá trị giảm trung bình: nan tỷ VND

Phân tích chu kỳ Jackpot:
- Số chu kỳ: 1
- Độ dài chu kỳ trung bình: 1356.00 kỳ
- Độ dài chu kỳ tối đa: 1356 kỳ
- Độ dài chu kỳ tối thiểu: 1356 kỳ


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import json
import os

# Tận dụng df từ cell trước
# Kiểm tra dữ liệu
if 'Kết Quả' not in df.columns:
    raise ValueError("Dữ liệu thiếu cột 'Kết Quả'")

# Chuyển đổi Kết Quả
df['Kết Quả'] = df['Kết Quả'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

# Kiểm tra tính hợp lệ
df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
if not invalid_results.empty:
    raise ValueError("Dữ liệu chứa kết quả không hợp lệ, vui lòng kiểm tra.")

# Mã hóa dữ liệu
mlb = MultiLabelBinarizer(classes=range(1, 46))
y_encoded = mlb.fit_transform(df['Kết Quả'])

# Tạo chuỗi time-series
sequence_length = 50
X, y = [], []
for i in range(len(y_encoded) - sequence_length):
    X.append(y_encoded[i:i + sequence_length])
    y.append(y_encoded[i + sequence_length])

X = np.array(X)  # (samples, sequence_length, 45)
y = np.array(y)  # (samples, 45)
print(f"Tổng số mẫu học: {len(X)}")

# Lưu dữ liệu mã hóa
np.save(os.path.join(OUTPUT_DIR, 'X.npy'), X)
np.save(os.path.join(OUTPUT_DIR, 'y.npy'), y)
print(f"Đã lưu dữ liệu mã hóa: X.npy, y.npy")

# Chia tập huấn luyện và kiểm tra
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

Tổng số mẫu học: 1306
Đã lưu dữ liệu mã hóa: X.npy, y.npy
Kích thước tập huấn luyện: (1044, 50, 45)
Kích thước tập kiểm tra: (262, 50, 45)


In [21]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv(r'E:\vietlot_mega\data_645/vietlott_645_clean.csv')

# Đếm số kỳ quay
num_draws = len(df)

print(f"Số kỳ quay trong dữ liệu: {num_draws}")

Số kỳ quay trong dữ liệu: 1356


In [ ]:
def load_and_preprocess_data(file_path='E:/vietlot_mega\data_645/vietlott_645_clean.csv'):
    logger.info(f"Đang tải dữ liệu từ {file_path}...")
    
    # Kiểm tra file tồn tại
    if not os.path.exists(file_path):
        logger.error(f"File {file_path} không tồn tại.")
        raise FileNotFoundError(f"File {file_path} không tồn tại.")
    
    # Đọc dữ liệu
    df = pd.read_csv(file_path, encoding='utf-8')
    if not all(col in df.columns for col in ['Ngày', 'Kết Quả']):
        logger.error("File CSV thiếu cột 'Ngày' hoặc 'Kết Quả'.")
        raise KeyError("File CSV thiếu cột cần thiết.")
    
    # Chuyển đổi cột Kết Quả
    def parse_results(res):
        try:
            if isinstance(res, str):
                numbers = json.loads(res) if '[' in res else [int(x) for x in res.split()]
            else:
                numbers = res
            if len(numbers) == 6 and all(isinstance(x, int) and 1 <= x <= 45 for x in numbers):
                return sorted(numbers)
            return None
        except Exception:
            return None
    
    initial_rows = len(df)
    df['Numbers'] = df['Kết Quả'].apply(parse_results)
    df = df.dropna(subset=['Numbers'])
    logger.info(f"Loại bỏ {initial_rows - len(df)} hàng do kết quả không hợp lệ.")
    
    # Chuyển đổi ngày
    df['Date'] = pd.to_datetime(df['Ngày'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')
    
    # Thêm đặc trưng
    df['Draw'] = range(1, len(df) + 1)
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Sum'] = df['Numbers'].apply(sum)
    df['OddCount'] = df['Numbers'].apply(lambda x: sum(1 for n in x if n % 2 == 1))
    df['Range'] = df['Numbers'].apply(lambda x: max(x) - min(x))
    
    logger.info(f"Đã tải {len(df)} kết quả hợp lệ.")
    return df

# Tải dữ liệu
df = load_and_preprocess_data()

2025-05-30 16:27:09,543 - INFO - Đang tải dữ liệu từ E:/vietlot_mega\data_645/vietlott_645_clean.csv...
2025-05-30 16:27:09,563 - INFO - Loại bỏ 0 hàng do kết quả không hợp lệ.
2025-05-30 16:27:09,569 - INFO - Đã tải 1356 kết quả hợp lệ.


# LSTM + GRU + Random Fores + Transformer + Tần xuất

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, LSTM, GRU, MultiHeadAttention, LayerNormalization, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.metrics import Precision
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import random
from collections import Counter
from datetime import datetime, timedelta
import os
import logging
import warnings

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler()])
warnings.filterwarnings('ignore')

# Thiết lập seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Cấu hình nâng cấp
CONFIG = {
    'file_path': 'E:\\vietlot_mega\\data_645\\vietlott_645_clean.csv',
    'sequence_length': 100,  # Tăng để học mẫu dài hơn
    'tcn_filters': [256, 128, 64],  # Tăng bộ lọc
    'lstm_units': [256, 128],  # Tăng đơn vị
    'gru_units': [256, 128],  # Tăng đơn vị
    'rf_estimators': 500,  # Tăng số cây
    'transformer_heads': 8,  # Tăng đầu chú ý
    'learning_rate': 0.0005,  # Giảm để ổn định
    'batch_size': 64,  # Tăng để xử lý nhanh
    'epochs': 200,  # Tăng để học kỹ hơn
    'dropout_rate': 0.3,
    'hist_ratio': 0.3
}

# Callback in số liệu huấn luyện
class PrintTrainingMetrics(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logging.info(f"Epoch {epoch+1}/{self.params['epochs']} - "
                     f"loss: {logs['loss']:.4f} - val_loss: {logs['val_loss']:.4f} - "
                     f"precision_at_k: {logs.get('precision_at_k', 0):.4f}")

class VietlottPredictor:
    def __init__(self, config=CONFIG):
        self.config = config
        self.df = None
        self.results = None
        self.scaler = MinMaxScaler()
        self.feature_scaler = MinMaxScaler()
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.last_sequence = None
        self.statistical_patterns = {}
        self.next_draw = None
        self.next_date = None

    def load_and_preprocess_data(self):
        """Tải và tiền xử lý dữ liệu Vietlott."""
        logging.info("Đang tải dữ liệu...")
        try:
            if not os.path.exists(self.config['file_path']):
                raise FileNotFoundError(f"File không tồn tại: {self.config['file_path']}")

            self.df = pd.read_csv(self.config['file_path'], encoding='utf-8')
            if 'Ngày' not in self.df.columns or 'Kết Quả' not in self.df.columns:
                raise ValueError("CSV phải chứa cột 'Ngày' và 'Kết Quả'")

            # Xử lý dữ liệu
            self.df['Numbers'] = self.df['Kết Quả'].apply(literal_eval)
            self.df['Date'] = pd.to_datetime(self.df['Ngày'], format='%Y-%m-%d')
            self.df = self.df.dropna(subset=['Date', 'Numbers']).sort_values('Date')

            # Lọc từ 2020
            self.df = self.df[self.df['Date'] >= '2020-01-01']
            if len(self.df) < 300:
                raise ValueError(f"Dữ liệu chỉ có {len(self.df)} kỳ, cần ít nhất 300 kỳ.")

            self.results = self.df['Numbers'].tolist()
            self.compute_statistical_patterns()

            # Tính đặc trưng
            self.df['OddCount'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if n % 2 == 1))
            self.df['Sum'] = self.df['Numbers'].apply(sum)
            self.df['Range'] = self.df['Numbers'].apply(lambda x: max(x) - min(x))
            self.df['RecentFreq'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['recent_frequencies'].get(n, 0) for n in x))
            self.df['PairFreq'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['pair_frequencies'].get(tuple(sorted([x[i], x[j]])), 0) for i in range(5) for j in range(i+1, 6)))

            logging.info(f"Dữ liệu đã tải: {len(self.df)} kỳ")
        except Exception as e:
            logging.error(f"Lỗi khi tải dữ liệu: {str(e)}")
            raise

    def compute_statistical_patterns(self):
        """Tính toán mẫu thống kê."""
        all_numbers = [num for draw in self.results for num in draw]
        freq = Counter(all_numbers)
        total_draws = len(self.results)
        self.statistical_patterns['frequencies'] = {int(k): v for k, v in freq.items()}
        self.statistical_patterns['number_probabilities'] = {int(k): v / (total_draws * 6) for k, v in freq.items()}
        self.statistical_patterns['hot_numbers'] = sorted(freq, key=freq.get, reverse=True)[:10] or list(range(1, 46))

        recent_numbers = [num for draw in self.results[-15:] for num in draw]
        recent_freq = Counter(recent_numbers)
        self.statistical_patterns['recent_frequencies'] = {int(k): v / 90 for k, v in recent_freq.items()} if recent_numbers else {i: 1/45 for i in range(1, 46)}

        pairs = [tuple(sorted([self.results[i][j], self.results[i][k]])) for i in range(len(self.results)) for j in range(5) for k in range(j+1, 6)]
        self.statistical_patterns['pair_frequencies'] = {p: v / total_draws for p, v in Counter(pairs).items()}

    def get_next_draw_and_date(self, steps_ahead=1):
        """Tính kỳ quay và ngày quay tiếp theo."""
        if self.df is None or self.df.empty:
            raise ValueError("Dữ liệu chưa được tải.")
        logging.info("Tính ngày quay tiếp theo...")

        self.df['Draw'] = range(1, len(self.df) + 1)
        last_draw = self.df['Draw'].iloc[-1]
        last_date = self.df['Date'].iloc[-1]

        days = ['Thứ Hai', 'Thứ Ba', 'Thứ Tư', 'Thứ Năm', 'Thứ Sáu', 'Thứ Bảy', 'Chủ Nhật']
        logging.info(f"Ngày cuối: {last_date.strftime('%Y-%m-%d')} ({days[last_date.weekday()]})")

        target_days = [2, 4, 6]  # Thứ Tư, Thứ Sáu, Chủ Nhật
        current_date = last_date
        for _ in range(steps_ahead):
            current_day = current_date.weekday()
            days_ahead = min((target - current_day) % 7 for target in target_days if target != current_day)
            if days_ahead == 0:
                days_ahead = min((target - current_day) % 7 or 7 for target in target_days if target != current_day)
            current_date += timedelta(days=days_ahead)

        self.next_date = current_date
        self.next_draw = last_draw + steps_ahead
        logging.info(f"Kỳ quay tiếp theo: {self.next_draw}, Ngày: {self.next_date.strftime('%Y-%m-%d')}")
        return self.next_draw, self.next_date

    def prepare_sequences(self):
        """Chuẩn bị chuỗi dữ liệu."""
        logging.info("Chuẩn bị chuỗi dữ liệu...")
        sequences = []
        targets = []
        features = []
        for i in range(len(self.results) - self.config['sequence_length']):
            seq = self.results[i:i + self.config['sequence_length']]
            target = self.results[i + self.config['sequence_length']]
            seq_features = [
                self.df['OddCount'].iloc[i:i + self.config['sequence_length']].values,
                self.df['Sum'].iloc[i:i + self.config['sequence_length']].values,
                self.df['Range'].iloc[i:i + self.config['sequence_length']].values,
                self.df['RecentFreq'].iloc[i:i + self.config['sequence_length']].values,
                self.df['PairFreq'].iloc[i:i + self.config['sequence_length']].values
            ]
            sequences.append(seq)
            targets.append(target)
            features.append(np.stack(seq_features, axis=-1))

        X = np.array(sequences)
        X_features = np.array(features)
        y = np.array(targets)

        X_scaled = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        X_features_scaled = self.feature_scaler.fit_transform(X_features.reshape(-1, X_features.shape[-1])).reshape(X_features.shape)
        X = np.concatenate([X_scaled, X_features_scaled], axis=-1)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=RANDOM_SEED, shuffle=False
        )

        last_sequence = np.array(self.results[-self.config['sequence_length']:])
        last_features = np.stack([
            self.df['OddCount'].iloc[-self.config['sequence_length']:].values,
            self.df['Sum'].iloc[-self.config['sequence_length']:].values,
            self.df['Range'].iloc[-self.config['sequence_length']:].values,
            self.df['RecentFreq'].iloc[-self.config['sequence_length']:].values,
            self.df['PairFreq'].iloc[-self.config['sequence_length']:].values
        ], axis=-1)
        last_sequence_scaled = self.scaler.transform(last_sequence.reshape(-1, 6)).reshape(1, self.config['sequence_length'], 6)
        last_features_scaled = self.feature_scaler.transform(last_features.reshape(-1, 5)).reshape(1, self.config['sequence_length'], 5)
        self.last_sequence = np.concatenate([last_sequence_scaled, last_features_scaled], axis=-1)

        logging.info(f"Tập huấn luyện: {self.X_train.shape}, Tập kiểm tra: {self.X_test.shape}")

    def build_tcn_model(self):
        """Xây dựng mô hình TCN."""
        model = Sequential([
            Conv1D(self.config['tcn_filters'][0], kernel_size=3, activation='relu', input_shape=(self.config['sequence_length'], 11), kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            Conv1D(self.config['tcn_filters'][1], kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            Conv1D(self.config['tcn_filters'][2], kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            Flatten(),
            Dense(45, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_lstm_model(self):
        """Xây dựng mô hình LSTM."""
        model = Sequential([
            LSTM(self.config['lstm_units'][0], input_shape=(self.config['sequence_length'], 11), return_sequences=True),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            LSTM(self.config['lstm_units'][1]),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            Dense(45, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_gru_model(self):
        """Xây dựng mô hình GRU."""
        model = Sequential([
            GRU(self.config['gru_units'][0], input_shape=(self.config['sequence_length'], 11), return_sequences=True),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            GRU(self.config['gru_units'][1]),
            BatchNormalization(),
            Dropout(self.config['dropout_rate']),
            Dense(45, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_transformer_model(self):
        """Xây dựng mô hình Transformer."""
        inputs = tf.keras.Input(shape=(self.config['sequence_length'], 11))
        x = MultiHeadAttention(num_heads=self.config['transformer_heads'], key_dim=64)(inputs, inputs)
        x = LayerNormalization(epsilon=1e-6)(x + inputs)
        x = Dense(128, activation='relu')(x)
        x = Dense(11)(x)
        x = LayerNormalization(epsilon=1e-6)(x + inputs)
        x = Flatten()(x)
        outputs = Dense(45, activation='sigmoid')(x)
        model = tf.keras.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_rf_model(self):
        """Xây dựng mô hình Random Forest."""
        return RandomForestClassifier(n_estimators=self.config['rf_estimators'], random_state=RANDOM_SEED)

    def evaluate_model(self, model, model_name, is_rf=False):
        """Đánh giá mô hình."""
        if is_rf:
            X_test_rf = self.X_test.reshape(self.X_test.shape[0], -1)
            pred_probs = np.array([model.predict_proba(X_test_rf)[i][:, 1] for i in range(45)]).T
        else:
            pred_probs = model.predict(self.X_test, verbose=0)

        pred = [sorted(np.argsort(p)[-6:][::-1] + 1) for p in pred_probs]
        pred = [self.adjust_prediction(p) for p in pred]

        y_true = self.y_test.astype(int)
        matches = [len(set(p).intersection(set(t))) for p, t in zip(pred, y_true)]
        match_3 = sum(1 for m in matches if m >= 3) / len(matches) if matches else 0
        match_4_plus = sum(1 for m in matches if m >= 4) / len(matches) if matches else 0
        logging.info(f"{model_name} - Match 3+: {match_3:.3f}, Match 4+: {match_4_plus:.3f}")
        return pred_probs

    def adjust_prediction(self, pred):
        """Điều chỉnh dự đoán."""
        pred = sorted(list(set(pred)))
        if len(pred) < 6:
            pred.extend(random.sample(self.statistical_patterns['hot_numbers'], 6 - len(pred)))

        if sum(pred) < 130 or sum(pred) > 140:
            pred = pred[:5] + [random.choice(self.statistical_patterns['hot_numbers'])]
        if sum(1 for n in pred if n % 2 == 1) not in [3, 4]:
            odd_count = sum(1 for n in pred if n % 2 == 1)
            candidates = [n for n in self.statistical_patterns['hot_numbers'] if n % 2 == (1 if odd_count < 3 else 0)]
            pred = pred[:5] + [random.choice(candidates or self.statistical_patterns['hot_numbers'])]
        if not (any(1 <= n <= 15 for n in pred) and any(16 <= n <= 30 for n in pred) and any(31 <= n <= 45 for n in pred)):
            pred = pred[:3] + [
                random.choice([n for n in self.statistical_patterns['hot_numbers'] if 1 <= n <= 15] or range(1, 16)),
                random.choice([n for n in self.statistical_patterns['hot_numbers'] if 16 <= n <= 30] or range(16, 31)),
                random.choice([n for n in self.statistical_patterns['hot_numbers'] if 31 <= n <= 45] or range(31, 46))
            ]
        return sorted(pred[:6])

    def train_models(self):
        """Huấn luyện các mô hình."""
        logging.info("Huấn luyện mô hình...")
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7),
            PrintTrainingMetrics()
        ]
        y_train_bin = np.array([[1 if j+1 in draw else 0 for j in range(45)] for draw in self.y_train])
        y_test_bin = np.array([[1 if j+1 in draw else 0 for j in range(45)] for draw in self.y_test])

        models = {}
        probs = {}
        for name, builder in [
            ('tcn', self.build_tcn_model),
            ('lstm', self.build_lstm_model),
            ('gru', self.build_gru_model),
            ('transformer', self.build_transformer_model)
        ]:
            logging.info(f"Huấn luyện {name}...")
            model = builder()
            model.fit(self.X_train, y_train_bin, validation_data=(self.X_test, y_test_bin),
                      epochs=self.config['epochs'], batch_size=self.config['batch_size'], callbacks=callbacks, verbose=0)
            probs[name] = self.evaluate_model(model, name)
            models[name] = model

        logging.info("Huấn luyện Random Forest...")
        rf_model = self.build_rf_model()
        X_train_rf = self.X_train.reshape(self.X_train.shape[0], -1)
        rf_model.fit(X_train_rf, y_train_bin)
        probs['rf'] = self.evaluate_model(rf_model, 'rf', is_rf=True)
        models['rf'] = rf_model

        return models, probs

    def predict(self):
        """Dự đoán bộ số."""
        logging.info("Dự đoán bộ số...")
        models, probs = self.train_models()

        model_probs = {}
        for name in ['tcn', 'lstm', 'gru', 'transformer']:
            model_probs[name] = models[name].predict(self.last_sequence, verbose=0)[0]
        last_sequence_rf = self.last_sequence.reshape(1, -1)
        model_probs['rf'] = np.array([models['rf'].predict_proba(last_sequence_rf)[i][:, 1][0] for i in range(45)])
        hist_probs = np.array([self.statistical_patterns['number_probabilities'].get(i+1, 0) for i in range(45)])
        recent_probs = np.array([self.statistical_patterns['recent_frequencies'].get(i+1, 0) for i in range(45)])

        final_probs = (0.25 * model_probs['tcn'] + 0.25 * model_probs['lstm'] + 0.2 * model_probs['gru'] +
                       0.2 * model_probs['transformer'] + 0.1 * model_probs['rf'] +
                       0.15 * 0.7 * hist_probs + 0.15 * 0.3 * recent_probs)
        final_probs /= final_probs.sum()

        predictions = {}
        for name in model_probs:
            pred = sorted(np.argsort(model_probs[name])[-6:][::-1] + 1)
            predictions[name] = self.adjust_prediction(pred)

        freq_nums = sorted(self.statistical_patterns['recent_frequencies'], key=self.statistical_patterns['recent_frequencies'].get, reverse=True)[:6]
        predictions['frequency'] = self.adjust_prediction(freq_nums)

        ensemble_nums = sorted(np.argsort(final_probs)[-6:][::-1] + 1)
        predictions['ensemble'] = self.adjust_prediction(ensemble_nums)

        all_numbers = [num for pred in predictions.values() for num in pred]
        number_counts = Counter(all_numbers)
        common_numbers = sorted([num for num, count in number_counts.items() if count >= 2])

        final_prediction = common_numbers[:6]
        if len(final_prediction) < 6:
            remaining = [n for n in self.statistical_patterns['hot_numbers'] if n not in final_prediction]
            final_prediction.extend(random.sample(remaining, 6 - len(final_prediction)))
        final_prediction = sorted(final_prediction[:6])

        logging.info(f"Dự đoán cuối cùng cho kỳ {self.next_draw} ({self.next_date.strftime('%Y-%m-%d')}): {final_prediction}")
        return {
            'final_prediction': final_prediction,
            'common_numbers': common_numbers,
            'predictions': predictions,
            'draw': self.next_draw,
            'date': self.next_date
        }

    def run(self, steps_ahead=1):
        """Chạy quy trình dự đoán."""
        logging.info("Bắt đầu dự đoán...")
        try:
            self.load_and_preprocess_data()
            self.get_next_draw_and_date(steps_ahead)
            self.prepare_sequences()
            results = self.predict()
            logging.info(f"Hoàn tất dự đoán cho kỳ {self.next_draw}")
            return results
        except Exception as e:
            logging.error(f"Lỗi: {str(e)}")
            raise

if __name__ == "__main__":
    if not os.path.exists(CONFIG['file_path']):
        print(f"Lỗi: File {CONFIG['file_path']} không tồn tại!")
    else:
        predictor = VietlottPredictor()
        try:
            results = predictor.run(steps_ahead=2)
            print(f"\nDỰ ĐOÁN CHO KỲ {results['draw']} ({results['date'].strftime('%Y-%m-%d')}):")
            print(f"Số dự đoán: {results['final_prediction']}")
            print(f"Số chung: {results['common_numbers']}")
            print("\nDỰ ĐOÁN TỪ MÔ HÌNH:")
            for model, pred in results['predictions'].items():
                print(f"{model.capitalize()}: {pred}")
        except Exception as e:
            print(f"Lỗi: {str(e)}")

2025-05-31 10:20:34,410 - INFO - Bắt đầu dự đoán...
2025-05-31 10:20:34,412 - INFO - Đang tải dữ liệu...
2025-05-31 10:20:34,458 - INFO - Dữ liệu đã tải: 821 kỳ
2025-05-31 10:20:34,459 - INFO - Tính ngày quay tiếp theo...
2025-05-31 10:20:34,461 - INFO - Ngày cuối: 2025-05-28 (Thứ Tư)
2025-05-31 10:20:34,461 - INFO - Kỳ quay tiếp theo: 823, Ngày: 2025-06-01
2025-05-31 10:20:34,463 - INFO - Chuẩn bị chuỗi dữ liệu...
2025-05-31 10:20:34,600 - INFO - Tập huấn luyện: (576, 100, 11), Tập kiểm tra: (145, 100, 11)
2025-05-31 10:20:34,603 - INFO - Dự đoán bộ số...
2025-05-31 10:20:34,604 - INFO - Huấn luyện mô hình...
2025-05-31 10:20:34,693 - INFO - Huấn luyện tcn...
2025-05-31 10:20:37,154 - INFO - Epoch 1/200 - loss: 3.6544 - val_loss: 3.2235 - precision_at_k: 0.1334
2025-05-31 10:20:37,975 - INFO - Epoch 2/200 - loss: 3.3581 - val_loss: 2.9957 - precision_at_k: 0.1768
2025-05-31 10:20:38,756 - INFO - Epoch 3/200 - loss: 3.1166 - val_loss: 2.7816 - precision_at_k: 0.2138
2025-05-31 10:20:39


DỰ ĐOÁN CHO KỲ 823 (2025-06-01):
Số dự đoán: [4, 5, 7, 10, 15, 19]
Số chung: [4, 5, 7, 10, 15, 19, 20, 35, 37, 41]

DỰ ĐOÁN TỪ MÔ HÌNH:
Tcn: [3, 24, 29, 34, 35, 41]
Lstm: [10, 15, 19, 20, 35, 41]
Gru: [4, 7, 15, 19, 36, 37]
Transformer: [5, 5, 10, 20, 21, 31]
Rf: [4, 7, 10, 27, 33, 40]
Frequency: [5, 7, 8, 9, 20, 37]
Ensemble: [4, 7, 19, 20, 25, 35]
